# Ansible Modules
---

Modules in Ansible are discrete units of code that perform specific functions (ex. provision a storage account, restart a VM, copy a file...). Ansible modules can be run from the command line or as a task in an Ansible Playbook. 
- They are the basic building blocks of an Ansible Playbook
- They are idempotent (i.e. any module run after the first will result in no change... think the 'Call Elevator' button)
- There are both built-in modules (provided by Ansible) and custom modules (written by third parties... or you!)
- Arguments can be passed into modules to affect how they function

Below is a diagram showing the breakdown of a task in Ansible. Note that the task is comprised of exactly one Ansible Module. Module and Task variables are discussed further in the Variables section.

![Task Breakdown](./notebook-files/task.png)

## Demos
---

**1. Follow these steps for all demos below:**

1. Run the code as is
2. Uncomment the second line or task and run it again
3. Comment the first line or task and run it again

> Note:
> - any differences in output between the three demos
> - the `ls` command proves the copied file does or does not exist

### Command line

Running Ansible Modules from the command line is uncommon and should only be done if needed. This need will often involve testing module functionality or idempotency.

In [ ]:
ansible localhost -m command -a "cp /home/cvx_admin_user/notebooks/playbooks/files/dummy.txt /tmp/dummy-copy.yml" -e ansible_connection=local
# ansible localhost -m command -a "rm                   /tmp/dummy-copy.yml" -e ansible_connection=local

# Show the copied or removed file
ls "/tmp/dummy-copy.yml"

### As tasks

In order to run an Ansible Module in a playbook, we must run it as a task. Below you can see an example playbook that now runs the `command` module as a task. Take note that the modules and their arguments are _exactly_ the same as they were on the command line. This means that both ways of running the module are equivalent and will have exactly the same results.

In [ ]:
wpb "ansible_copy_remove1.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  tasks:
    - name: Copy the file using the 'command' module
      command: "cp /home/cvx_admin_user/notebooks/playbooks/files/dummy.txt /tmp/dummy-copy.yml"

    # - name: Delete the file using the 'command' module
    #   command: "rm /tmp/dummy-copy.yml"
...
############################################################

ansible-playbook "ansible_copy_remove1.yml"

ls "/tmp/dummy-copy.yml"

### For specific functions

Anytime you have a specific function in mind, be sure to check all the available Ansible Modules for one that that meets your needs. For example, instead of passing `cp` into the `command` module as an argument, we can run the `copy` module and achieve the same result.

In [ ]:
wpb "ansible_copy_remove2.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  tasks:
    - name: Copy the file using the 'copy' module
      copy:
        src: "/home/cvx_admin_user/notebooks/playbooks/files/dummy.txt"
        dest: "/tmp/dummy-copy2.yml"

    # - name: Delete the file using the 'file' module
    #   file:
    #     path: "/tmp/dummy-copy2.yml"
    #     state: absent
...
############################################################

ansible-playbook "ansible_copy_remove2.yml"
/bin/ls "/tmp/dummy-copy2.yml"

### Examples

Copy a file in place, for example a Message Of The Day (motd) file. Commonly used as a legal / security warning. 

> - Create a file under files directory
> - Use copy module to copy it to destination
> - If you logged into the machine, you will see motd displayed

In [ ]:
cat << EOF > "/home/cvx_admin_user/notebooks/playbooks/files/motd"

**************************************************************************

NOTICE TO USERS: This Chevron system is for use by authorized users only.
To the extent permitted by law, by using this system you acknowledge and
consent to the monitoring, access, use, and disclosure by Chevron of any
information generated, received, or stored on the system and waive any right
or expectation of privacy in connection with your use. Unauthorized and/or
improper use of this system in violation of Chevron corporate policies is
not tolerated and may result in disciplinary action, including termination.

**************************************************************************

EOF

In [ ]:
wpb "ansible_copy_motd.yml"

########################  PLAYBOOK  ########################
---
- hosts: localhost
  connection: local

  tasks:

  - name: Copy the file using the 'copy' module
    copy:
      src: "/home/cvx_admin_user/notebooks/playbooks/files/motd"
      dest: "/etc/motd"
      owner: "root"
      group: "root"
      mode: 0644
    become: true

...
############################################################

ansible-playbook "ansible_copy_motd.yml"

/bin/ls -l /etc/motd
cat /etc/motd

## Further Explanation
---

**Example Module Usage:**
``` yaml
# Copy a file using a built-in Ansible module
- name: Copy
  copy:
    src: "dummy.txt"
    dest: "/tmp/dummy-copy.txt"
```

**Notes:**
> - When running modules on the command line:
>   - `-a` specifies the arguments getting passed into the module
>   - `-e` specifies the extra-vars
>   - the source of the copyt can be local or a remote file (with remote_src: true set)
>   - the source can be relative or absolute.  src will look under files directory in relative as well as local directory.
>   - by default the copy module looks in the local directory and in the files subdirectory
> - Anyone can write their own custom Ansible Module in any language, it just has to return a JSON object

**See Also:**
- [Intro to Modules](https://docs.ansible.com/ansible/latest/user_guide/modules_intro.html)
- [List of Modules](https://docs.ansible.com/ansible/latest/collections/index_module.html)